# `quickmt` Backtranslation Notebook

In [1]:
import math
from itertools import islice
from pathlib import Path

import datasets
import numpy as np
from sacrebleu import BLEU, CHRF
from tqdm import tqdm

from quickmt.translator import Translator

/home/mark/miniforge3/envs/eole/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
src_lang = "fa"

In [6]:
# For GPU inference
qmt = Translator(
    f"/home/mark/mt/quickmt-models/quickmt-en-{src_lang}",
    device="cuda",
    intra_threads=1,
    inter_threads=4,
)

In [7]:
def batch(iterable, n):
    "Batch data into lists of length n. The last batch may be shorter."
    # batched('ABCDEFG', 3) --> ABC DEF G
    it = iter(iterable)
    while True:
        batch = tuple(islice(it, n))
        if not batch:
            return
        yield batch

In [8]:
src_file = "/home/mark/mt/data/madlad.en"
tgt_file = f"/home/mark/mt/data/madlad.backtrans.{src_lang}"

In [ ]:
# 6851it [10:40:52,  5.61s/it]
with open(src_file, "rt") as myfile:
    with open(tgt_file, "wt") as outfile:
        for b in tqdm(batch(myfile, 10000)):
            src = [i.strip() for i in b]
            mt = qmt(src, beam_size=4, max_batch_size=80)
            outfile.write("\n".join(mt) + "\n")


3301it [5:26:38,  5.99s/it]

In [ ]:
src_file = "/home/mark/mt/data/newscrawl.2024.en"
tgt_file = f"/home/mark/mt/data/newscrawl.backtrans.{src_lang}"

In [ ]:
# 3510it [8:50:45,  9.07s/it]
with open(src_file, "rt") as myfile:
    with open(tgt_file, "wt") as outfile:
        for b in tqdm(batch(myfile, 10000)):
            src = [i.strip() for i in b]
            mt = qmt(src, beam_size=4, max_batch_size=80)
            outfile.write("\n".join(mt) + "\n")
